In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import edward as ed
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
print(tf.__version__)
from edward.models import Multinomial, Normal

1.1.0


In [3]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('../VAE/Datasets/MNIST_data', one_hot=True)
n_train_samples = mnist.train.num_examples
n_test_samples = mnist.test.num_examples
print(n_train_samples)
print(n_test_samples)

Extracting ../VAE/Datasets/MNIST_data/train-images-idx3-ubyte.gz
Extracting ../VAE/Datasets/MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../VAE/Datasets/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../VAE/Datasets/MNIST_data/t10k-labels-idx1-ubyte.gz
55000
10000


### Define the model

In [5]:
def forward(X, W_0, b_0, W_1, b_1):
  hidden_layer = tf.nn.relu(tf.matmul(X, W_0) + b_0)
  output = tf.matmul(hidden_layer, W_1) + b_1
  return output

### Priors

In [9]:
num_features = 784
num_classes = 10
hidden_units = 500

W_0 = Normal(loc=tf.zeros([num_features, hidden_units]), scale=tf.ones([num_features, hidden_units]))
b_0 = Normal(loc=tf.zeros(hidden_units), scale=tf.ones(hidden_units))
W_1 = Normal(loc=tf.zeros([hidden_units, num_classes]), scale=tf.ones([hidden_units, num_classes]))
b_1 = Normal(loc=tf.zeros(num_classes), scale=tf.ones(num_classes))

In [ ]:
X = tf.placeholder(tf.float32, [N, num_features])